# 필요 라이브러리 설치

In [57]:
!pip install transformers datasets accelerate scikit-learn optuna sentencepiece

In [58]:
!pip install transformers

In [85]:
import pandas as pd
import numpy as np
from sklearn.metrics import f1_score, classification_report
from sklearn.preprocessing import LabelEncoder
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from datasets import Dataset, DatasetDict
import torch
import optuna
from sklearn.model_selection import train_test_split
from tqdm import tqdm
import re
import platform
import transformers
import joblib

In [86]:
# GPU 사용 여부 확인
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

Using device: cuda


# 사용할 데이터

In [87]:
from google.colab import drive
drive.mount('/content/gdrive')
!ln -s /content/gdrive/My\ Drive/ /mydrive
!ls /mydrive

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
ln: failed to create symbolic link '/mydrive/My Drive': File exists
'Colab Notebooks'  'My Drive'   신문기사_original.pkl   이전   졸업논문


In [88]:
import pandas as pd
train=pd.read_csv('/content/gdrive/MyDrive/졸업논문/신문기사_불균형데이터_train.csv', encoding='utf-8-sig')

In [112]:
train['category'].value_counts()

,count
category,
정치,5000
경제,4000
사회,2500
"IT,과학",500
교육/입시/NIE,100


In [89]:
import pandas as pd
test=pd.read_csv('/content/gdrive/MyDrive/졸업논문/신문기사_불균형데이터_test.csv', encoding='utf-8-sig')

#데이터 정제

In [90]:
def clean_korean_documents(documents):

    # 텍스트 정제 (특수기호만 제거)
    for i, document in tqdm(enumerate(documents), total=len(documents), desc="Cleaning Special Characters"):
        document = re.sub(r'[^\w\sㄱ-ㅣ가-힣]', '', document)  # 특수기호만 제거, 정규 표현식
        documents[i] = document

    return documents

def remove_words_with_numbers(text):
    return ' '.join([word for word in text.split(',') if not re.search(r'\d', word)])

# 한글이 아닌 문자를 제거하는 함수
def remove_non_korean(text):
    return re.sub(r'[^ㄱ-ㅣ가-힣\s]', '', text)

In [91]:
train['text'] = clean_korean_documents(train['text'])
train['text'] = train['text'].apply(lambda x: ','.join(x.split()))
train['text'] = train['text'].apply(remove_words_with_numbers)
train['text'] = train['text'].apply(remove_non_korean)

test['text'] = clean_korean_documents(test['text'])
test['text'] = test['text'].apply(lambda x: ','.join(x.split()))
test['text'] = test['text'].apply(remove_words_with_numbers)
test['text'] = test['text'].apply(remove_non_korean)

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  documents[i] = document
<ipython-input-90-681e93e9a2a0>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  documents[i] = document
<ipython-input-90-681e93e9a2a0>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  documents[i] = document
<ipython-input-90-681e93e9a2a0>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user

In [92]:
# 3. 학습 데이터와 테스트 데이터에서 필요한 열 추출 및 라벨 인코딩
X_train = train['text']
y_train = train['category']
X_test = test['text']
y_test=test['category']

In [93]:
# 4. 라벨 인코딩
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded=label_encoder.transform(y_test)
num_labels = len(label_encoder.classes_)

In [94]:
# 5. KoBERT 모델을 위한 토크나이저 및 데이터셋 생성
model_name = 'monologg/kobert'
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)

In [95]:
# 텍스트를 토큰화하는 함수 정의
def tokenize_function(examples):
    return tokenizer(examples['text'], padding='max_length', truncation=True, max_length=256)

In [96]:
# 데이터셋 생성 및 토크나이즈
train_df = pd.DataFrame({'text': X_train, 'label': y_train_encoded})
test_df = pd.DataFrame({'text': X_test, 'label': y_test_encoded})
train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)
train_dataset = train_dataset.map(tokenize_function, batched=True)
test_dataset = test_dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/12100 [00:00<?, ? examples/s]

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

In [97]:
# PyTorch 텐서 형식으로 변환
train_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])
test_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])

# 5. KoBERT 모델 설정 및 학습 파라미터 최적화
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=num_labels).to(device)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at monologg/kobert and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


# 고정된 파라미터로 학습하기

In [98]:
#CPU 측정

import psutil
import time
from transformers import Trainer, TrainingArguments

# CPU 사용량 및 속도 측정 함수
def measure_cpu_usage_and_time(train_function):
    start_time = time.time()  # 시작 시간 기록
    start_cpu = psutil.cpu_percent(interval=None)  # 시작 CPU 사용량

    train_function()  # 학습 함수 호출

    end_time = time.time()  # 종료 시간 기록
    end_cpu = psutil.cpu_percent(interval=None)  # 종료 CPU 사용량

    elapsed_time = end_time - start_time
    avg_cpu_usage = (start_cpu + end_cpu) / 2

    print(f"Elapsed Time: {elapsed_time:.2f} seconds")
    print(f"Average CPU Usage: {avg_cpu_usage:.2f}%")

In [99]:
import optuna
from transformers import Trainer, TrainingArguments
from sklearn.metrics import f1_score
import numpy as np

# `compute_metrics` 함수
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    f1 = f1_score(labels, predictions, average='macro')  # F1 점수 계산
    return {'eval_f1': f1}  # 'eval_f1' 반환

# Optuna를 활용한 하이퍼파라미터 튜닝
def model_init():
    return AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=num_labels)

In [100]:
# 7. 최적의 파라미터로 학습 설정
# 최적의 하이퍼파라미터를 찾은 후, training_args를 설정합니다.
training_args = TrainingArguments(
    output_dir='/content',
    num_train_epochs=10,  # 에포크 수 고정
    per_device_train_batch_size=64,  # 배치 크기 고정
    learning_rate=5.0e-05,  # 최적화된 학습률
    weight_decay=0.01,  # 최적화된 weight decay
    evaluation_strategy='epoch',
    save_strategy='epoch',
    logging_dir='/content',
    logging_steps=10,
    fp16=True,
    disable_tqdm=False,
    load_best_model_at_end=True,
    report_to="none"
)

# 최적의 파라미터로 Trainer 구성
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=num_labels)

trainer7 = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,  # 전체 학습 데이터로 평가
    compute_metrics=compute_metrics
)

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at monologg/kobert and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [101]:
# CPU 사용량 및 속도 측정
measure_cpu_usage_and_time(trainer7.train)

Epoch,Training Loss,Validation Loss,F1
1,0.474300,0.934012,0.540539
2,0.405000,0.763648,0.685443
3,0.226300,0.855813,0.724554
4,0.148500,1.024610,0.684612
5,0.144800,0.854326,0.759853
6,0.050800,1.103935,0.751982
7,0.037200,1.357073,0.708732
8,0.015900,1.291183,0.753886
9,0.014600,1.365392,0.749085
10,0.001700,1.452831,0.737723


Elapsed Time: 1502.86 seconds
Average CPU Usage: 11.45%


In [102]:
# 모델 저장
joblib.dump(trainer7, '/content/gdrive/MyDrive/신문기사_original.pkl')

print("모델이 저장되었습니다.")

모델이 저장되었습니다.


In [103]:
# 8. 저장된 모델로 테스트 데이터 예측 및 결과 저장
# 저장된 모델 불러오기

loaded_model = joblib.load('/content/gdrive/MyDrive/신문기사_original.pkl')

predictions7 = loaded_model.predict(test_dataset)

test_predictions7 = np.argmax(predictions7.predictions, axis=1)
decoded_test_predictions7 = label_encoder.inverse_transform(test_predictions7)

In [104]:
predictions7 = trainer7.predict(test_dataset)
test_predictions7 = np.argmax(predictions7.predictions, axis=1)
decoded_test_predictions7 = label_encoder.inverse_transform(test_predictions7)

In [105]:
test["category_prediction"] = decoded_test_predictions7

selected_columns7 = test[['text', 'category', 'category_prediction']]
selected_columns7.head(3)

,text,category,category_prediction
0,스마트폰 중독이 부른 사고 주제 청소년 경연대회 참가 김정대 기자 광주 산정초등학교...,정치,사회
1,직무수행능력 공직 가치관 등과 관련한 종합의견 제시대구도시공사 청렴도 향상 및 변화...,정치,사회
2,무등육아원서 강연체험부스 운영 광주 동구명장명인장인협회가 지난 무등육아원에서 첫 번...,정치,사회


In [106]:
selected_columns7 = selected_columns7.rename(columns={"category_prediction": "y_pred"})
selected_columns7 = selected_columns7.rename(columns={"category": "y_true"})

In [107]:
selected_columns7.to_csv('/content/신문기사_original_prediction.csv', encoding='utf-8-sig')

# 평가

In [108]:
from sklearn.metrics import f1_score

import pandas as pd

df=pd.read_csv('/content/신문기사_original_prediction.csv', encoding='utf-8-sig')

df.head(3)

,Unnamed: 0,text,y_true,y_pred
0,0,스마트폰 중독이 부른 사고 주제 청소년 경연대회 참가 김정대 기자 광주 산정초등학교...,정치,사회
1,1,직무수행능력 공직 가치관 등과 관련한 종합의견 제시대구도시공사 청렴도 향상 및 변화...,정치,사회
2,2,무등육아원서 강연체험부스 운영 광주 동구명장명인장인협회가 지난 무등육아원에서 첫 번...,정치,사회


In [109]:
train['category'].value_counts()

,count
category,
정치,5000
경제,4000
사회,2500
"IT,과학",500
교육/입시/NIE,100


In [110]:
from sklearn.metrics import classification_report, f1_score

# y_true와 y_pred를 추출
y_true = df['y_true']
y_pred = df['y_pred']

# classification_report를 사용하여 각 클래스별 F1-score 출력
report = classification_report(y_true, y_pred, digits=4)
print(report)

              precision    recall  f1-score   support

       IT,과학     0.8947    0.4930    0.6357      1000
          경제     0.6368    0.8310    0.7210      1000
   교육/입시/NIE     0.9613    0.3970    0.5619      1000
          사회     0.5187    0.8440    0.6426      1000
          정치     0.8252    0.9110    0.8660      1000

    accuracy                         0.6952      5000
   macro avg     0.7673    0.6952    0.6854      5000
weighted avg     0.7673    0.6952    0.6854      5000

